<a href="https://colab.research.google.com/github/JuanGHub/MLCourseCCC/blob/main/MLA_NLP_DAY1_BOW_NB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <a name="0">Machine Learning Accelerator - Natural Language Processing - Lecture 1</a>

## Bag of Words Method

In this notebook, we go over the Bag of Words (BoW) method to convert text data into numerical values, that will be later used for predictions with machine learning algorithms.

To convert text data to vectors of numbers, a vocabulary of known words (tokens) is extracted from the text, the occurence of words is scored, and the resulting numerical values are saved in vocabulary-long vectors. There are a few versions of BoW, corresponding to different words scoring methods. We use the Sklearn library to calculate the BoW numerical values using:

1. <a href="#1">Binary</a>
2. <a href="#2">Word Counts</a>
3. <a href="#3">Term Frequencies</a>
4. <a href="#4">Term Frequency-Inverse Document Frequencies</a>


In [1]:
# Upgrade dependencies
#!pip install -q -r ../../requirements.txt

In [14]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

## 1. <a name="1">Binary</a>
(<a href="#0">Go to top</a>)

Let's calculate the first type of BoW, recording whether the word is in the sentence or not. We will also go over some useful features of Sklearn's vectorizers here.


In [4]:
sentences = [
    "This document is the first document",
    "This document is the second document",
    "and this is the third one",
]

# Initialize the count vectorizer with the parameter: binary=True
binary_vectorizer = CountVectorizer(binary=True)

# fit_transform() function fits the text data and gets the binary BoW vectors
x = binary_vectorizer.fit_transform(sentences)

As the vocabulary size grows, the BoW vectors also get very large in size. They are usually made of many zeros and very few non-zero values. Sklearn stores these vectors in a compressed form. If we want to use them as Numpy arrays, we call the __toarray()__ function. Here are our binary BoW features. Each row corresponds to a single document.

In [5]:
x.toarray()

array([[0, 1, 1, 1, 0, 0, 1, 0, 1],
       [0, 1, 0, 1, 0, 1, 1, 0, 1],
       [1, 0, 0, 1, 1, 0, 1, 1, 1]])

Let's check out our vocabulary. We can use the __vocabulary___ attribute. This returns a dictionary with each word as key and index as value. Notice that the indices are assigned in alphabetical ordered.

In [7]:
binary_vectorizer.vocabulary_

{'this': 8,
 'document': 1,
 'is': 3,
 'the': 6,
 'first': 2,
 'second': 5,
 'and': 0,
 'third': 7,
 'one': 4}

Similar information can be reached with the __get_feature_names_out()__ function. The position of the terms in the .get_feature_names() correspond to the column position of the elements in the BoW matrix.

In [8]:
print(binary_vectorizer.get_feature_names_out())

['and' 'document' 'first' 'is' 'one' 'second' 'the' 'third' 'this']


We can easily compute the total number of times that each of the words from the vocabulary appears in the corpus.

In [9]:
sum_words = x.sum(axis=0)
words_freq = [
    (word, sum_words[0, idx])
    for (idx, word) in enumerate(binary_vectorizer.get_feature_names_out())
]
words_freq

[('and', np.int64(1)),
 ('document', np.int64(2)),
 ('first', np.int64(1)),
 ('is', np.int64(3)),
 ('one', np.int64(1)),
 ('second', np.int64(1)),
 ('the', np.int64(3)),
 ('third', np.int64(1)),
 ('this', np.int64(3))]

Here're the binary BoW vectors associated to each of the sentences of the corpus.

In [11]:
df = pd.DataFrame(
    x.toarray(), columns=binary_vectorizer.get_feature_names_out(), index=sentences
)
df

,and,document,first,is,one,second,the,third,this
This document is the first document,0,1,1,1,0,0,1,0,1
This document is the second document,0,1,0,1,0,1,1,0,1
and this is the third one,1,0,0,1,1,0,1,1,1


In [12]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=df)

MessageError: Error: credential propagation was unsuccessful

How can we calculate BoW for a new text? We will use the __transform()__ function this time. You can see below this doesn't change the vocabulary. New words are simply skipped in this case.

In [13]:
new_sentence = ["This is the new sentence"]

new_vectors = binary_vectorizer.transform(new_sentence)

In [15]:
new_vectors.toarray()

array([[0, 0, 0, 1, 0, 0, 1, 0, 1]])

In [16]:
df2 = pd.DataFrame(
    new_vectors.toarray(),
    columns=binary_vectorizer.get_feature_names_out(),
    index=new_sentence,
)
pd.concat([df, df2])

,and,document,first,is,one,second,the,third,this
This document is the first document,0,1,1,1,0,0,1,0,1
This document is the second document,0,1,0,1,0,1,1,0,1
and this is the third one,1,0,0,1,1,0,1,1,1
This is the new sentence,0,0,0,1,0,0,1,0,1


## 2. <a name="2">Word Counts</a>
(<a href="#0">Go to top</a>)

Word counts can be simply calculated using the same __CountVectorizer()__ function __without__ the __binary__ parameter.



In [17]:
sentences = [
    "This document is the first document",
    "This document is the second document",
    "and this is the third one",
]

# Initialize the count vectorizer
count_vectorizer = CountVectorizer()

xc = count_vectorizer.fit_transform(sentences)

xc.toarray()

array([[0, 2, 1, 1, 0, 0, 1, 0, 1],
       [0, 2, 0, 1, 0, 1, 1, 0, 1],
       [1, 0, 0, 1, 1, 0, 1, 1, 1]])

In [18]:
df = pd.DataFrame(
    xc.toarray(), columns=binary_vectorizer.get_feature_names_out(), index=sentences
)
df

,and,document,first,is,one,second,the,third,this
This document is the first document,0,2,1,1,0,0,1,0,1
This document is the second document,0,2,0,1,0,1,1,0,1
and this is the third one,1,0,0,1,1,0,1,1,1


In [19]:
new_sentence = ["This is the new sentence"]
new_vectors = count_vectorizer.transform(new_sentence)
new_vectors.toarray()

array([[0, 0, 0, 1, 0, 0, 1, 0, 1]])

In [20]:
df2 = pd.DataFrame(
    new_vectors.toarray(),
    columns=binary_vectorizer.get_feature_names_out(),
    index=new_sentence,
)
pd.concat([df, df2])

,and,document,first,is,one,second,the,third,this
This document is the first document,0,2,1,1,0,0,1,0,1
This document is the second document,0,2,0,1,0,1,1,0,1
and this is the third one,1,0,0,1,1,0,1,1,1
This is the new sentence,0,0,0,1,0,0,1,0,1


## 3. <a name="3">Term Frequency (TF)</a>
(<a href="#0">Go to top</a>)

Term Frequency (TF) vectors that show how important words are to the documents, are computed using

$$tf(term, doc) = \frac{\text{number of times that the term occurs in the doc}}{\text{total number of terms in the doc}}$$

From `sklearn` we use the `TfidfVectorizer` function with the parameter`use_idf=False`, which additionally *automatically normalizes* the term frequencies vectors by their Euclidean ($L_2$) norm.


In [21]:
tf_vectorizer = TfidfVectorizer(use_idf=False)

x = tf_vectorizer.fit_transform(sentences)

np.round(x.toarray(), 2)

array([[0.  , 0.71, 0.35, 0.35, 0.  , 0.  , 0.35, 0.  , 0.35],
       [0.  , 0.71, 0.  , 0.35, 0.  , 0.35, 0.35, 0.  , 0.35],
       [0.41, 0.  , 0.  , 0.41, 0.41, 0.  , 0.41, 0.41, 0.41]])

In [22]:
new_sentence = ["This is the new sentence"]
new_vectors = tf_vectorizer.transform(new_sentence)
np.round(new_vectors.toarray(), 2)

array([[0.  , 0.  , 0.  , 0.58, 0.  , 0.  , 0.58, 0.  , 0.58]])

In [23]:
df = pd.DataFrame(
    np.round(x.toarray(), 2), columns=tf_vectorizer.get_feature_names_out(), index=sentences
)
df2 = pd.DataFrame(
    np.round(new_vectors.toarray(), 2),
    columns=tf_vectorizer.get_feature_names_out(),
    index=new_sentence,
)
pd.concat([df, df2])

,and,document,first,is,one,second,the,third,this
This document is the first document,0.00,0.71,0.35,0.35,0.00,0.00,0.35,0.00,0.35
This document is the second document,0.00,0.71,0.00,0.35,0.00,0.35,0.35,0.00,0.35
and this is the third one,0.41,0.00,0.00,0.41,0.41,0.00,0.41,0.41,0.41
This is the new sentence,0.00,0.00,0.00,0.58,0.00,0.00,0.58,0.00,0.58


## 4. <a name="4">Term Frequency Inverse Document Frequency (TF-IDF)</a>
(<a href="#0">Go to top</a>)

Term Frequency Inverse Document Frequency (TF-IDF) vectors are computed using the `TfidfVectorizer()` function with the parameter `use_idf=True`. We can also skip this parameter as it is already `True` by default.


In [24]:
tfidf_vectorizer = TfidfVectorizer(use_idf=True)

sentences = [
    "This document is the first document",
    "This document is the second document",
    "and this is the third one",
]

xf = tfidf_vectorizer.fit_transform(sentences)

np.round(xf.toarray(), 2)

array([[0.  , 0.73, 0.48, 0.28, 0.  , 0.  , 0.28, 0.  , 0.28],
       [0.  , 0.73, 0.  , 0.28, 0.  , 0.48, 0.28, 0.  , 0.28],
       [0.5 , 0.  , 0.  , 0.29, 0.5 , 0.  , 0.29, 0.5 , 0.29]])

In [25]:
new_sentence = ["This is the new sentence"]
new_vectors = tfidf_vectorizer.transform(new_sentence)
np.round(new_vectors.toarray(), 2)

array([[0.  , 0.  , 0.  , 0.58, 0.  , 0.  , 0.58, 0.  , 0.58]])

In [26]:
df = pd.DataFrame(
    np.round(xf.toarray(), 2),
    columns=tfidf_vectorizer.get_feature_names_out(),
    index=sentences,
)
df2 = pd.DataFrame(
    np.round(new_vectors.toarray(), 2),
    columns=tfidf_vectorizer.get_feature_names_out(),
    index=new_sentence,
)
pd.concat([df, df2])

,and,document,first,is,one,second,the,third,this
This document is the first document,0.0,0.73,0.48,0.28,0.0,0.00,0.28,0.0,0.28
This document is the second document,0.0,0.73,0.00,0.28,0.0,0.48,0.28,0.0,0.28
and this is the third one,0.5,0.00,0.00,0.29,0.5,0.00,0.29,0.5,0.29
This is the new sentence,0.0,0.00,0.00,0.58,0.0,0.00,0.58,0.0,0.58


__Note 1__: In addition to *automatically normalizing* the term frequencies vectors by their Euclidean ($L_2$) norm, sklearn also uses a *smoothed version of idf*, computing

$$idf(term) = \ln \Big( \frac{n_{documents} +1}{n_{documents\,containing\,the\,term}+1}\Big) + 1$$

In [27]:
np.round(tfidf_vectorizer.idf_, 2)

array([1.69, 1.29, 1.69, 1.  , 1.69, 1.69, 1.  , 1.69, 1.  ])

Here we can see how the IDF is larger for the less common terms and viceversa.

In [28]:
df = pd.DataFrame(
    [[str(a) for a in np.round(tfidf_vectorizer.idf_, 2)]],
    columns=tfidf_vectorizer.get_feature_names_out(),
    index=["IDF"],
)
df2 = pd.DataFrame(
    [[str(w[1]) for w in words_freq]],
    columns=tfidf_vectorizer.get_feature_names_out(),
    index=["Term Frequency"],
)
pd.concat([df2, df])

,and,document,first,is,one,second,the,third,this
Term Frequency,1,2,1,3,1,1,3,1,3
IDF,1.69,1.29,1.69,1.0,1.69,1.69,1.0,1.69,1.0
